<a href="https://colab.research.google.com/github/kruspe2009/volunteer_projects/blob/main/DonorSearch/DonorSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание проекта


Сервис DonorSearch поставил перед собой 2 цели: 

1.   Увеличить количество доноров
2.   Больше вовлекать существующих доноров

В качестве исходных данных предоставлены 3 таблицы:



*   Донации крови
*   Планируемые донации крови
*   Анонимная таблица доноров




# Импорт библиотек и чтение данных

In [1]:
import pandas as pd
import numpy as np

In [2]:
donations_anon = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DonorSearch/donations_anon.csv')
plan_anon = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DonorSearch/plan_anon.csv')
users_anon = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DonorSearch/users_anon.csv')

# Знакомство с данными



## Предобработка данных

## Таблица `donations_anon`

In [3]:
#изучим данные в таблице
donations_anon.head()

,ID,ID пользователя,Класс крови,Дата донации,Дата добавления донации,Тип донации,Регион,Место стадчи,Статус донации,Есть справка
0,97920,151894,Плазма,02.06.2021,02.06.2021,Безвозмездно,"Россия, Москва",647,На модерации,Да
1,97919,156916,Тромбоциты,02.06.2021,02.06.2021,Безвозмездно,"Россия, Москва",633,На модерации,Да
2,97918,149413,Цельная кровь,01.06.2021,02.06.2021,Безвозмездно,"Россия, Приморский край, Владивосток",325,Принята,Да
3,97917,155027,Цельная кровь,02.06.2021,02.06.2021,Безвозмездно,"Россия, Саратовская область, Саратов",500,Принята,Да
4,97916,151094,Цельная кровь,23.04.2021,02.06.2021,Безвозмездно,"Россия, Свердловская область, Екатеринбург",509,Принята,Да


In [4]:
#посмотрим на столбцы
donations_anon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97874 entries, 0 to 97873
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID                       97874 non-null  int64 
 1   ID пользователя          97874 non-null  int64 
 2   Класс крови              97874 non-null  object
 3   Дата донации             97874 non-null  object
 4   Дата добавления донации  97874 non-null  object
 5   Тип донации              97874 non-null  object
 6   Регион                   97874 non-null  object
 7   Место стадчи             97874 non-null  object
 8   Статус донации           97874 non-null  object
 9   Есть справка             97874 non-null  object
dtypes: int64(2), object(8)
memory usage: 7.5+ MB


В процессе чтения таблицы с данными о донациях, возникла проблема с парсингом даты - выяснилось, что некоторые даты указаны в формате, отличном от стандартного для данного датасета. Посмотрим на них более подробно. 

In [5]:
#"стандарт" для данного датасета - 10 символов в дате
dates = set()

#отберем даты, отличные от стандарта
for date in donations_anon['Дата донации']:
  if len(date) != 10:
    dates.add(date)
    print(date)

21.04.208
27.04.201
15.12.214
24.05.207


In [6]:
#уберем некорректные данные из датасета
donations_anon = donations_anon[~donations_anon['Дата донации'].isin(dates)]

In [7]:
#по столбцу "Дата добавления донации" проблем нет
for date in donations_anon['Дата добавления донации']:
  if len(date) != 10:
    dates.add(date)
    print(date)

При попытке преобразовать дату донации из строкового формата в формат даты возвращается ошибка. По всей вероятности, помимо даты, в которой меньше символов, чем в стандартной дате, в некоторых строках указаны некорректные даты.

In [8]:
date_list = set()

for date in donations_anon['Дата донации']:
  date_split = date.split('.')
  for i in date_split:
    if int(date_split[2]) > 2021 or int(date_split[2]) < 1980:
      date_list.add(date)

In [9]:
#снова уберем некорректные данные из датасета
donations_anon = donations_anon[~donations_anon['Дата донации'].isin(date_list)]

In [10]:
#преобразование дат из строкового типа в тип даты
donations_anon['Дата донации'] = pd.to_datetime(donations_anon['Дата донации'], format='%d.%m.%Y')
donations_anon['Дата добавления донации'] = pd.to_datetime(donations_anon['Дата добавления донации'], format='%d.%m.%Y')

In [11]:
#посмотрим на дипазон дат добавления донации, так как с датой самой донации ранее мы уже обнаружили проблемы
print('Минимальная дата добавления донации:', donations_anon['Дата добавления донации'].min())
print('Максимальная дата добавления донации:',  donations_anon['Дата добавления донации'].max())

Минимальная дата добавления донации: 2020-11-18 00:00:00
Максимальная дата добавления донации: 2021-06-02 00:00:00


In [12]:
#методом value_counts смотрим уникальные значения во всех столбцах на предмет наличия каких-то некорректных категорийных переменных
donations_anon['Регион'].value_counts()

Не указано                                                          92109
Россия, Москва                                                       1885
Россия, Татарстан, Казань                                             514
Россия, Санкт-Петербург                                               493
Россия, Татарстан, Набережные Челны                                   163
                                                                    ...  
Россия, Самарская область, Сызрань                                      1
Россия, Ханты-Мансийский Автономный округ - Югра АО, Нефтеюганск        1
Россия, Московская область, Фрязино                                     1
Казахстан, Кзыл-Ординская область, Байконур                             1
Россия, Северная Осетия - Алания, Владикавказ                           1
Name: Регион, Length: 267, dtype: int64

In [13]:
#проверим, нет ли в сете дублирующих строк
print('Количество полных дубликатов в данных:', sum(donations_anon.duplicated()))

Количество полных дубликатов в данных: 0


Контекст процесса подразумевает, что событие "донация крови" на временной плоскости лежит до его регистрации внутри системы DonorSearch. Исходя из этой посылки, мы делаем предположение, что дата добавления донации не может быть меньше, чем дата непосредственно донации. Если записи, не удовлетворяющие данному условию в датасете все таки будут обнаружены, то их нельзя признать достоверными и лучше удалить при небольшом количестве "ошибок".

In [14]:
#выведем  некорректные записи
donations_anon[donations_anon['Дата добавления донации'] < donations_anon['Дата донации']]

,ID,ID пользователя,Класс крови,Дата донации,Дата добавления донации,Тип донации,Регион,Место стадчи,Статус донации,Есть справка
271,97647,158384,Плазма,2021-05-21,2021-05-20,Безвозмездно,"Россия, Тульская область, Тула",556,Без справки,Нет
770,97147,158054,Цельная кровь,2021-04-30,2021-04-29,Безвозмездно,"Россия, Свердловская область, Екатеринбург",509,Без справки,Нет
929,96988,157962,Цельная кровь,2021-04-26,2021-04-25,Безвозмездно,"Россия, Московская область, Одинцово",684,Принята,Да
1133,96784,157785,Цельная кровь,2021-04-21,2021-04-20,Безвозмездно,Не указано,567,Без справки,Нет
2261,95656,156669,Цельная кровь,2021-03-13,2021-03-12,Платно,"Россия, Москва",1027,Удалена,Нет
2360,95557,156538,Тромбоциты,2021-03-10,2021-03-09,Платно,"Россия, Москва",863,Без справки,Нет
2526,95391,156458,Цельная кровь,2021-03-05,2021-03-04,Безвозмездно,"Россия, Санкт-Петербург",658,Без справки,Нет
2724,95193,2080,Цельная кровь,2021-02-25,2021-02-24,Безвозмездно,"Россия, Камчатский край, Петропавловск-Камчатский",113,Принята,Да
2742,95175,155991,Эритроциты,2021-02-24,2021-02-23,Платно,"Россия, Москва",649,Без справки,Нет
2757,95160,155991,Эритроциты,2021-02-22,2021-02-21,Платно,"Россия, Москва",649,Удалена,Нет


In [15]:
print('Количество записей с некорректным расположением процедуры донации и ее регистрации на временной оси:', len(donations_anon[donations_anon['Дата добавления донации'] < donations_anon['Дата донации']]))

Количество записей с некорректным расположением процедуры донации и ее регистрации на временной оси: 27


In [16]:
#27 записей для целей исследования определяющего значения не имеют, поэтому уберем эти записи из датасета
donations_anon = donations_anon[donations_anon['Дата добавления донации'] >= donations_anon['Дата донации']]

Мы установили, что наибольшая потребность в предобработке в столбце `Регион`, так как в нем через запятую содержатся топонимы, характеризующие место процедуры. Для возможного последующего анализа географии мы разделим записи в столбце на два столбца: страна и город проведения процедуры. 

Первичный визуальный анализ показывает, что до первой запятой всегда следует название страны, а после последней запятой указано наименование города, поэтому разделим топонимы, исходя из этого принципа.

In [17]:
#заранее сформируем в датафрейме пустые столбцы, куда запишем наименование страны и города донации
donations_anon['country'] = np.nan
donations_anon['city'] = np.nan

In [18]:
#при помощи функции мы разберем локацию на составляющик
def topo_parser(row):
  if row['Регион'] != 'Не указано':
    parsed_topo = row['Регион'].split(',')
    row['country'] = parsed_topo[0]
    row['city'] = parsed_topo[-1]
  else:
    row['country'] = row['Регион']
    row['city'] = row['Регион']
  return row

In [19]:
donations_anon = donations_anon.apply(topo_parser, axis=1)

In [20]:
donations_anon.head()

,ID,ID пользователя,Класс крови,Дата донации,Дата добавления донации,Тип донации,Регион,Место стадчи,Статус донации,Есть справка,country,city
0,97920,151894,Плазма,2021-06-02,2021-06-02,Безвозмездно,"Россия, Москва",647,На модерации,Да,Россия,Москва
1,97919,156916,Тромбоциты,2021-06-02,2021-06-02,Безвозмездно,"Россия, Москва",633,На модерации,Да,Россия,Москва
2,97918,149413,Цельная кровь,2021-06-01,2021-06-02,Безвозмездно,"Россия, Приморский край, Владивосток",325,Принята,Да,Россия,Владивосток
3,97917,155027,Цельная кровь,2021-06-02,2021-06-02,Безвозмездно,"Россия, Саратовская область, Саратов",500,Принята,Да,Россия,Саратов
4,97916,151094,Цельная кровь,2021-04-23,2021-06-02,Безвозмездно,"Россия, Свердловская область, Екатеринбург",509,Принята,Да,Россия,Екатеринбург


Функция отработала корректно - теперь оценим пропуски, то есть у какого количества наблюдений нет страны и города проведения донации.

In [21]:
donations_anon['country'].value_counts()

Не указано    92104
Россия         5565
Украина          70
Беларусь         25
Казахстан        24
Молдова           4
Кыргызстан        4
Name: country, dtype: int64

У большей части записей изначально не определен регион, но в нашем распоряжении идентификатор места сдачи. Попробуем на основании имеющихся данных установить, где расположена каждая из станций.

In [22]:
print('Количество уникальных идентификаторов места сдачи в полном датасете:', len(donations_anon['Место стадчи'].unique()))
print('Количество уникальных идентификаторов места сдачи в датасете с указанным регионом:', len(donations_anon[donations_anon['Регион'] != 'Не указано']['Место стадчи'].unique()))

Количество уникальных идентификаторов места сдачи в полном датасете: 776
Количество уникальных идентификаторов места сдачи в датасете с указанным регионом: 338


In [23]:
#заранее убедимся, что для каждого идентификатора доступен только один вариант страны/города
places_topo = donations_anon[donations_anon['Регион'] != 'Не указано'].groupby('Место стадчи', as_index=False).agg({'country': 'nunique', 'city': 'nunique'}).sort_values('city', ascending=False)
places_topo = places_topo.rename(columns={'Место стадчи': 'place'})
places_topo

,place,country,city
337,Выездная акция,3,74
229,656,1,2
223,650,1,2
222,649,1,1
230,657,1,1
...,...,...,...
111,387,1,1
110,386,1,1
109,382,1,1
108,378,1,1


Во-первых, одним из "мест" указана некая "Выездная акция". По всей видимости, такой формат и не должен иметь никакой конкретной локации.
Во-вторых, в данных действительно обнаружились разночтения с географической локацией для некоторых мест сдачи. 

In [24]:
places = places_topo[(places_topo['place'] != 'Выездная акция') & (places_topo['city'] > 1)]
places

,place,country,city
229,656,1,2
223,650,1,2


In [25]:
donations_anon[donations_anon['Место стадчи'] == '650']['city'].value_counts()

Не указано     121
 Зеленоград      9
 Москва          1
Name: city, dtype: int64

In [26]:
donations_anon[donations_anon['Место стадчи'] == '656']['city'].value_counts()

Не указано      1637
 Москва          110
 Краснокамск       1
Name: city, dtype: int64

In [27]:
donations_anon.head()

,ID,ID пользователя,Класс крови,Дата донации,Дата добавления донации,Тип донации,Регион,Место стадчи,Статус донации,Есть справка,country,city
0,97920,151894,Плазма,2021-06-02,2021-06-02,Безвозмездно,"Россия, Москва",647,На модерации,Да,Россия,Москва
1,97919,156916,Тромбоциты,2021-06-02,2021-06-02,Безвозмездно,"Россия, Москва",633,На модерации,Да,Россия,Москва
2,97918,149413,Цельная кровь,2021-06-01,2021-06-02,Безвозмездно,"Россия, Приморский край, Владивосток",325,Принята,Да,Россия,Владивосток
3,97917,155027,Цельная кровь,2021-06-02,2021-06-02,Безвозмездно,"Россия, Саратовская область, Саратов",500,Принята,Да,Россия,Саратов
4,97916,151094,Цельная кровь,2021-04-23,2021-06-02,Безвозмездно,"Россия, Свердловская область, Екатеринбург",509,Принята,Да,Россия,Екатеринбург


In [28]:
#снова формируем датафрейм с локациями, отсеяв двойственные значения функцие max
places_topo = donations_anon[donations_anon['Регион'] != 'Не указано'].groupby('Место стадчи', as_index=False).agg({'country': 'max', 'city': 'max'})
places_topo = places_topo.rename(columns={'Место стадчи': 'place'})
places_topo

,place,country,city
0,1,Россия,Барнаул
1,100,Россия,Иркутск
2,101,Россия,Братск
3,1017,Россия,Чебоксары
4,102,Россия,Усолье-Сибирское
...,...,...,...
333,987,Россия,Находка
334,99,Россия,Южа
335,990,Россия,Кирово-Чепецк
336,992,Россия,Северск


In [29]:
#откинем столбцы со страной и городом из основного датафрейма, так как они нам уже не нужны
donations_anon = donations_anon.drop(['country', 'city'], axis=1)

In [30]:
donations_anon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97796 entries, 0 to 97873
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ID                       97796 non-null  int64         
 1   ID пользователя          97796 non-null  int64         
 2   Класс крови              97796 non-null  object        
 3   Дата донации             97796 non-null  datetime64[ns]
 4   Дата добавления донации  97796 non-null  datetime64[ns]
 5   Тип донации              97796 non-null  object        
 6   Регион                   97796 non-null  object        
 7   Место стадчи             97796 non-null  object        
 8   Статус донации           97796 non-null  object        
 9   Есть справка             97796 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 8.2+ MB


In [31]:
#при помощи левого соединения добавим данные из таблицы с локациями станций к основному фрейму
donations_anon = donations_anon.merge(places_topo, left_on='Место стадчи', right_on='place', how='left')
donations_anon = donations_anon.drop(['place'], axis=1)

In [32]:
donations_anon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97796 entries, 0 to 97795
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ID                       97796 non-null  int64         
 1   ID пользователя          97796 non-null  int64         
 2   Класс крови              97796 non-null  object        
 3   Дата донации             97796 non-null  datetime64[ns]
 4   Дата добавления донации  97796 non-null  datetime64[ns]
 5   Тип донации              97796 non-null  object        
 6   Регион                   97796 non-null  object        
 7   Место стадчи             97796 non-null  object        
 8   Статус донации           97796 non-null  object        
 9   Есть справка             97796 non-null  object        
 10  country                  89228 non-null  object        
 11  city                     89228 non-null  object        
dtypes: datetime64[ns](2), int64(2), 

In [33]:
donations_anon['country'] = donations_anon['country'].fillna('Не указано')
donations_anon['city'] = donations_anon['city'].fillna('Не указано')

In [36]:
donations_anon['country'].value_counts()

Россия        77871
Украина       10382
Не указано     8568
Казахстан       520
Беларусь        337
Молдова         114
Кыргызстан        4
Name: country, dtype: int64

Если ранее локация конкретной донации была не установлена у порядка 92 тысяч записей, то теперь таковых осталось всего 8500, что составляет порядка 8.8% от всего числа записей (ранее не установлена локация была у 94.2% записей).